## ⅍ Joins

#### 1. 🔗 Подключаемся к базе данных

In [32]:
import pandas as pd
import sqlite3

db_path = "checking-logs.sqlite"
conn = sqlite3.connect(db_path)

#### 2. Создаём временную таблицу и выполняем SQL-запрос 

In [33]:
query = """
CREATE TEMP TABLE datamart AS
SELECT
    c.uid,
    c.labname,
    c.timestamp AS first_commit_ts,
    MIN(p.datetime) AS first_view_ts
FROM checker AS c
LEFT JOIN pageviews AS p ON c.uid = p.uid
WHERE c.status = 'ready'
AND c.numTrials = 1
AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname;
"""
conn.execute(query)

#### 3. Загружаем таблицу datamart в Pandas DataFrame

In [34]:
datamart = pd.read_sql("SELECT * FROM datamart;", conn)
print(datamart.head())

      uid  labname             first_commit_ts               first_view_ts
0  user_1   laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1  user_1  laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2  user_1   laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3  user_1   laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4  user_1  laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136


#### 4. Преобразование столбцов в формат datetime

In [35]:
datamart["first_commit_ts"] = pd.to_datetime(datamart["first_commit_ts"])
datamart["first_view_ts"] = pd.to_datetime(datamart["first_view_ts"])
print(datamart.head())

      uid  labname            first_commit_ts              first_view_ts
0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3  user_1   laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4  user_1  laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136


#### 5. Разделение на тестовую и контрольную группы

In [36]:
test = datamart[datamart["first_view_ts"].notna()].copy()
control = datamart[datamart["first_view_ts"].isna()].copy()
print(test.head())
print(control.head())

      uid  labname            first_commit_ts              first_view_ts
0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3  user_1   laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4  user_1  laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136
        uid   labname            first_commit_ts first_view_ts
12  user_11    laba05 2020-05-03 21:06:55.970293           NaT
13  user_11  project1 2020-05-03 23:45:33.673409           NaT
14  user_12    laba04 2020-04-18 17:07:51.767358           NaT
15  user_12   laba04s 2020-04-26 15:42:38.070593           NaT
16  user_12    laba05 2020-05-03 08:39:25.174316           NaT


#### 6. Заполнение пропущенных значений в контрольной группе

In [37]:
mean_first_view_ts = test["first_view_ts"].mean()
control.fillna({"first_view_ts": mean_first_view_ts}, inplace=True)
print(control.head())

        uid   labname            first_commit_ts                 first_view_ts
12  user_11    laba05 2020-05-03 21:06:55.970293 2020-04-27 00:40:05.761783552
13  user_11  project1 2020-05-03 23:45:33.673409 2020-04-27 00:40:05.761783552
14  user_12    laba04 2020-04-18 17:07:51.767358 2020-04-27 00:40:05.761783552
15  user_12   laba04s 2020-04-26 15:42:38.070593 2020-04-27 00:40:05.761783552
16  user_12    laba05 2020-05-03 08:39:25.174316 2020-04-27 00:40:05.761783552


#### 7. Сохранение тестовой и контрольной групп в базу данных и вывод рез-тов

In [38]:
test.to_sql("test", conn, if_exists="replace", index=False)
control.to_sql("control", conn, if_exists="replace", index=False)

conn.close()

print("\nПредварительный просмотр таблицы datamart:")
print(datamart.head())

print("\nПредварительный просмотр тестовой группы:")
print(test.head())

print("\nПредварительный просмотр контрольной группы (с заменой пропущенных значений):")
print(control.head())


Предварительный просмотр таблицы datamart:
      uid  labname            first_commit_ts              first_view_ts
0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3  user_1   laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4  user_1  laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136

Предварительный просмотр тестовой группы:
      uid  labname            first_commit_ts              first_view_ts
0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3  user_1   laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4  user_1  laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136

Предварительный просмотр контрольной